In [8]:
class Box:

    def __str__(self):
        return f"Box(xmin={self.xmin}, xmax={self.xmax}, "\
                   f"ymin={self.ymin}, ymax={self.ymax})"
    
    def __init__(self, xmin, xmax, ymin, ymax):
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax

In [7]:
import Konstanten as k
print(k.m+k.q)

51.0
